# Lab 3.1 – Agent with Watsonx.ai (Google Colab Version)

---

## 🚀 Run this notebook in Google Colab

**Prerequisites:**
- IBM Cloud API Key ([Create one here](https://cloud.ibm.com/iam/apikeys))
- Watsonx Project ID ([Find it in your watsonx.ai project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp))

This notebook demonstrates building an **AI Agent** that can:
- Use multiple tools (RAG service, calculator)
- Plan which tool to use based on user input
- Generate natural language responses

**High-level flow:**
1. User asks a question
2. Granite model acts as a **planner** (chooses which tool to use)
3. Python executes the selected tool
4. Granite generates a **final answer** based on tool output

---

## Step 1: Install Dependencies

In [ ]:
# Install required dependencies
!pip install -qU "ibm-watsonx-ai>=1.1.22"
!pip install -qU requests
!pip install -qU pydantic

print("✅ All dependencies installed successfully!")

## Step 2: Configure Credentials

In [ ]:
import os
import json
import time
import ast
import operator as op
import getpass
from typing import Dict, Any

# Prompt for Watsonx credentials
WATSONX_APIKEY = getpass.getpass("Enter IBM Cloud API Key: ")
WATSONX_PROJECT_ID = getpass.getpass("Enter Watsonx Project ID: ")
WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
LLM_MODEL_ID = "ibm/granite-3-3-8b-instruct"  # You can change this to other Granite models

print("\n✅ Credentials configured!")

## Step 3: Initialize Watsonx Model

We'll use the Granite model as our planner and answer generator.

In [ ]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

creds = Credentials(url=WATSONX_URL, api_key=WATSONX_APIKEY)

params = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 512,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.2,
}

planner_model = Model(
    model_id=LLM_MODEL_ID,
    credentials=creds,
    project_id=WATSONX_PROJECT_ID,
    params=params,
)

print("✅ Watsonx Granite model initialized!")

## Step 4: Define Tools

Our agent will have access to two tools:

1. **RAG Service Tool** - Mock implementation (you can connect to a real RAG API)
2. **Calculator Tool** - Safely evaluates arithmetic expressions

### RAG Service Tool (Mock)

⚠️ **Note:** In the original notebook, this connects to a FastAPI `/ask` endpoint. For Colab, we'll use a mock implementation. You can replace this with a real API call if you have a deployed RAG service.

In [ ]:
import requests

def rag_service_tool(question: str) -> Dict[str, Any]:
    """
    Mock RAG service. Replace with actual API call if you have a deployed RAG endpoint.
    
    Example:
        ACCELERATOR_API_URL = "http://your-rag-api.com/ask"
        payload = {"question": question}
        resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
        resp.raise_for_status()
        return resp.json()
    """
    # Mock response for demonstration
    mock_responses = {
        "what is rag": {
            "answer": "RAG (Retrieval Augmented Generation) is a technique that combines information retrieval with language model generation. It retrieves relevant documents from a knowledge base and uses them to generate more accurate and contextual responses.",
            "citations": ["RAG Documentation"],
            "model_id": "mock-model",
            "latency_ms": 150
        },
        "default": {
            "answer": "I don't have specific information about that in my knowledge base. This is a mock RAG response. In a real implementation, this would query your RAG API endpoint.",
            "citations": [],
            "model_id": "mock-model",
            "latency_ms": 100
        }
    }
    
    # Simple keyword matching for demo
    question_lower = question.lower()
    if "rag" in question_lower:
        return mock_responses["what is rag"]
    else:
        return mock_responses["default"]

print("✅ RAG service tool defined (using mock implementation)")

### Calculator Tool

Safe arithmetic calculator using AST parsing.

In [ ]:
# Safe calculator implementation using AST
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.Mod: op.mod,
}

def _eval_ast(node):
    if isinstance(node, ast.Num):  # type: ignore[attr-defined]
        return node.n
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")

def calculator_tool(expression: str) -> str:
    """
    Safely evaluate a simple arithmetic expression.
    Supports +, -, *, /, %, and exponentiation.
    """
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"

print("✅ Calculator tool defined")

## Step 5: Define Planner Schema

The planner decides which tool to use based on the user input.

In [ ]:
from pydantic import BaseModel

class ToolPlan(BaseModel):
    tool: str
    arguments: Dict[str, Any]

PLANNER_SYSTEM_PROMPT = (
    "You are a planner agent. You must choose exactly ONE tool per request.\n\n"
    "Available tools:\n"
    "- rag_service: Use this to answer enterprise questions using the RAG API.\n"
    "- calculator: Use this to evaluate arithmetic expressions like '2 * (3 + 4)'.\n\n"
    "Return a JSON object with keys 'tool' and 'arguments'. Do not include any extra text.\n"
    "If the user is clearly asking a math question, prefer 'calculator'. Otherwise, prefer 'rag_service'."
)

print("✅ Planner schema defined")

## Step 6: Implement Planner Function

In [ ]:
def plan_tool_call(user_input: str) -> ToolPlan:
    """Ask the LLM which tool to use and with what arguments."""
    user_prompt = (
        f"User input: {user_input}\n\n"
        "Respond ONLY with JSON, for example:\n"
        '{"tool": "calculator", "arguments": {"expression": "2 + 2"}}'
    )
    prompt = f"{PLANNER_SYSTEM_PROMPT}\n\n{user_prompt}"
    
    raw = planner_model.generate_text(prompt=prompt)
    text = raw.strip()
    
    # Try to extract JSON
    try:
        plan_dict = json.loads(text)
    except json.JSONDecodeError:
        # Fallback: try to find JSON substring
        start = text.find("{")
        end = text.rfind("}") + 1
        if start >= 0 and end > start:
            plan_dict = json.loads(text[start:end])
        else:
            raise ValueError(f"Could not parse JSON from planner output: {text!r}")
    
    return ToolPlan(**plan_dict)

print("✅ Planner function implemented")

## Step 7: Implement Final Answer Generator

In [ ]:
FINAL_ANSWER_SYSTEM = (
    "You are a helpful assistant. You will be given: the original user question, the tool you used, "
    "and the tool output. Compose a clear and concise final answer. "
    "If the tool output indicates an error, explain the error and suggest what the user can try."
)

def generate_final_answer(user_input: str, tool_name: str, tool_output: Any) -> str:
    """Call the LLM to turn tool output into a final answer."""
    context = (
        f"User question: {user_input}\n"
        f"Tool used: {tool_name}\n"
        f"Tool output: {tool_output}\n"
        "\nPlease write the final answer for the user."
    )
    prompt = f"{FINAL_ANSWER_SYSTEM}\n\n{context}"
    
    raw = planner_model.generate_text(prompt=prompt)
    return raw.strip()

print("✅ Final answer generator implemented")

## Step 8: Orchestrate the Agent

In [ ]:
def run_agent_once(user_input: str) -> Dict[str, Any]:
    """
    Run one turn of the agent:
    1. Plan which tool to use
    2. Execute the tool
    3. Generate final answer
    """
    # Step 1: Plan
    print(f"\n🤔 Planning for: {user_input}")
    plan = plan_tool_call(user_input)
    tool_name = plan.tool
    args = plan.arguments or {}
    
    print(f"🔧 Selected tool: {tool_name}")
    print(f"📋 Arguments: {args}")
    
    # Step 2: Execute tool
    if tool_name == "rag_service":
        question = args.get("question") or user_input
        tool_output = rag_service_tool(question)
    elif tool_name == "calculator":
        expr = args.get("expression") or user_input
        tool_output = calculator_tool(expr)
    else:
        tool_output = f"Unknown tool: {tool_name}"
    
    print(f"⚙️  Tool output: {tool_output}")
    
    # Step 3: Generate final answer
    print("\n✍️  Generating final answer...")
    final_answer = generate_final_answer(user_input, tool_name, tool_output)
    
    return {
        "question": user_input,
        "tool": tool_name,
        "tool_args": args,
        "tool_output": tool_output,
        "final_answer": final_answer,
    }

print("✅ Agent orchestration implemented")

## Step 9: Test the Agent

Let's test with a few different types of questions.

### Test 1: RAG Question

In [ ]:
question_1 = "What is RAG and why do we use it?"
result_1 = run_agent_once(question_1)

print("\n" + "=" * 80)
print("FINAL RESULT")
print("=" * 80)
print(f"Question: {result_1['question']}")
print(f"Tool used: {result_1['tool']}")
print(f"\nFinal Answer:\n{result_1['final_answer']}")
print("=" * 80)

### Test 2: Math Question

In [ ]:
question_2 = "What is 2 * (3 + 4)?"
result_2 = run_agent_once(question_2)

print("\n" + "=" * 80)
print("FINAL RESULT")
print("=" * 80)
print(f"Question: {result_2['question']}")
print(f"Tool used: {result_2['tool']}")
print(f"\nFinal Answer:\n{result_2['final_answer']}")
print("=" * 80)

### Test 3: Another Math Question

In [ ]:
question_3 = "Calculate 15 * 8 + 42"
result_3 = run_agent_once(question_3)

print("\n" + "=" * 80)
print("FINAL RESULT")
print("=" * 80)
print(f"Question: {result_3['question']}")
print(f"Tool used: {result_3['tool']}")
print(f"\nFinal Answer:\n{result_3['final_answer']}")
print("=" * 80)

## Step 10: Try Your Own Questions!

In [ ]:
your_question = input("Enter your question: ")
result = run_agent_once(your_question)

print("\n" + "=" * 80)
print("FINAL RESULT")
print("=" * 80)
print(f"Question: {result['question']}")
print(f"Tool used: {result['tool']}")
print(f"Tool arguments: {result['tool_args']}")
print(f"\nFinal Answer:\n{result['final_answer']}")
print("=" * 80)

---

## Summary

You successfully completed this notebook! You learned how to:

✅ Build an AI agent using Watsonx Granite models  
✅ Implement tool selection via LLM planning  
✅ Create custom tools (RAG service, calculator)  
✅ Orchestrate multi-step agent workflows  
✅ Generate natural language responses from tool outputs  

**Next Steps:**
- Add more tools (web search, database query, etc.)
- Connect the RAG service to a real API endpoint
- Implement multi-turn conversations
- Add error handling and retries
- Experiment with different Granite models

**Connecting to a Real RAG API:**

To connect to a real RAG endpoint, modify the `rag_service_tool` function:

```python
def rag_service_tool(question: str) -> Dict[str, Any]:
    ACCELERATOR_API_URL = "http://your-rag-api.com/ask"
    payload = {"question": question}
    start = time.time()
    resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
    latency_ms = int((time.time() - start) * 1000)
    resp.raise_for_status()
    data = resp.json()
    data.setdefault("latency_ms", latency_ms)
    return data
```

For more information:
- [Watsonx.ai Documentation](https://ibm.github.io/watsonx-ai-python-sdk/samples.html)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)

---

**Copyright © 2024 IBM. This notebook and its source code are released under the terms of the MIT License.**